In [1]:
from qdrant_client import QdrantClient
import os
from dotenv import load_dotenv
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.cohere import CohereEmbedding

from llama_index.core import Settings
load_dotenv()

embedding = CohereEmbedding(
    model="embed-english-v3.0",
    api_key=os.getenv("COHERE_API_KEY")
)

Settings.embed_model = embedding

client=QdrantClient(url="http://localhost:6333")
vector_store = QdrantVectorStore(client=client, collection_name="training_examples", show_progress=True)
storage_context=StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(
vector_store,
storage_context=storage_context,
)


In [2]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(api_key=os.getenv("COHERE_API_KEY"),top_n=3)

retriver=index.as_retriever(similarity_top_k=3)
advanced_retriever=index.as_retriever(similarity_top_k=10)

In [18]:
query_str="Go to the fridge"
response=retriver.retrieve(query_str)
advanced_response=advanced_retriever.retrieve(query_str)
cohere_rerank_response=cohere_rerank.postprocess_nodes(query_str=query_str, nodes=advanced_response)

In [19]:
print("Query: ", query_str)
print("\nKNN response")
print(response[0].text)
print("Advanced response")
print(cohere_rerank_response[0].text)

Query:  Go to the fridge

KNN response

goal: I am at the living room and I want to go to the counter.
Answer: Go straight passed the couch, turn left and go straight and wait near the counter. 

Advanced response

goal: I am at the living room and I want to go to the counter.
Answer: Go straight passed the couch, turn left and go straight and wait near the counter. 

